In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

from src.features import build_features

In [31]:
df = pd.read_csv(r'C:\Project_MB\Anomaly-Detection-of-Air-Compressor\data\raw\train_data.csv')
test = pd.read_csv(r'C:\Project_MB\Anomaly-Detection-of-Air-Compressor\data\raw\test_data.csv')

In [32]:
df = build_features.create_derived_features(df)
test = build_features.create_derived_features(test)

In [33]:
df

,air_inflow,air_end_temp,out_pressure,motor_current,motor_rpm,motor_temp,motor_vibe,type,air_flow_pressure,current_by_airflow,temp_diff,current_by_vibration,current_by_rpm,temp_by_vibration,rpm_diff_vibration,current_temp_diff,vibration_rpm_product,current_by_airflow_ratio,vibration_endtemp_product,rpm_airflow_product
0,1.59,41.00,0.7,20.53,1680.0,58.67,2.93,0,1.113,12.911950,17.67,60.1529,0.012220,120.1300,1677.07,-20.47,4922.40,12.911950,120.1300,2671.20
1,2.97,59.28,0.7,38.40,3142.0,74.91,3.75,0,2.079,12.929293,15.63,144.0000,0.012222,222.3000,3138.25,-20.88,11782.50,12.929293,222.3000,9331.74
2,1.91,45.29,0.7,24.73,2023.0,62.48,3.12,0,1.337,12.947644,17.19,77.1576,0.012224,141.3048,2019.88,-20.56,6311.76,12.947644,141.3048,3863.93
3,2.37,51.33,0.7,30.63,2506.0,67.84,3.39,0,1.659,12.924051,16.51,103.8357,0.012223,174.0087,2502.61,-20.70,8495.34,12.924051,174.0087,5939.22
4,1.90,45.21,0.7,24.65,2017.0,62.41,3.12,0,1.330,12.973684,17.20,76.9080,0.012221,141.0552,2013.88,-20.56,6293.04,12.973684,141.0552,3832.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2458,2.28,50.20,0.7,29.53,2416.0,66.84,3.34,7,1.596,12.951754,16.64,98.6302,0.012223,167.6680,2412.66,-20.67,8069.44,12.951754,167.6680,5508.48
2459,2.04,46.94,0.7,26.34,2155.0,63.94,3.20,7,1.428,12.911765,17.00,84.2880,0.012223,150.2080,2151.80,-20.60,6896.00,12.911765,150.2080,4396.20
2460,1.19,35.74,0.7,15.39,1259.0,53.99,2.70,7,0.833,12.932773,18.25,41.5530,0.012224,96.4980,1256.30,-20.35,3399.30,12.932773,96.4980,1498.21
2461,1.21,36.00,0.7,15.64,1280.0,54.22,2.71,7,0.847,12.925620,18.22,42.3844,0.012219,97.5600,1277.29,-20.36,3468.80,12.925620,97.5600,1548.80


In [34]:
from sklearn.ensemble import IsolationForest

# 모델 학습 및 이상치 탐지
model = IsolationForest(n_estimators=1000, max_samples='auto', contamination=0.06, random_state=42)
model.fit(df)

c:\Users\kki96\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


IsolationForest(contamination=0.06, n_estimators=1000, random_state=42)

In [35]:
test_score = test.copy()
test_score["anomaly_score"] = model.decision_function(test_score)

In [36]:
test_score["anomaly"] = test_score["anomaly_score"].apply(lambda x: 0 if x > 0 else 1)


In [37]:
test_score.anomaly.value_counts()

0    7038
1     351
Name: anomaly, dtype: int64

In [40]:
submission = pd.read_csv(r'C:\Project_MB\Anomaly-Detection-of-Air-Compressor\data\raw\answer_sample.csv')
submission

,type,label
0,0,-1
1,0,-1
2,0,-1
3,0,-1
4,0,-1
...,...,...
7384,7,-1
7385,7,-1
7386,7,-1
7387,7,-1


In [41]:
submission["label"] = test_score["anomaly"]

In [43]:
submission.to_csv(r'C:\Project_MB\Anomaly-Detection-of-Air-Compressor\data\submission\ML.csv', index=False)